## Testovanie sentimentu pomocou ChatGPT – Few-shot prompting

In [ ]:
# Inštalácia knižníc
!pip install openai pandas scikit-learn matplotlib seaborn
     

import pandas as pd
import openai # nainštalovanie openai, lebo Kaggle ho nemá predinštalovaný
import time
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt

        

## 1. Načítanie dát

In [ ]:
# Dataset so 1000 komentármi (500 toxických + 500 netoxických)
df = pd.read_csv("/kaggle/input/chatgpt/chatgpt_eval_1000_slovak.csv")


##  2. Tvorba few-shot promptu

In [ ]:
import os
import openai


from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("OPENAI_API_KEY") # použivam systémovú premennú 
                        
client = openai.OpenAI(api_key=api_key)


# 
# client = openai.OpenAI(api_key="")  # vlastný API kľúč, nepoužívať
                           

def create_prompt(text):
    return (
        "Je nasledujúci slovenský komentár toxický? Odpovedz iba 'Áno' alebo 'Nie'.\n\n"
        "Príklad 1:\n"
        "Komentár: \"Si úplný idiot, zmizni!\"\n"
        "Odpoveď: Áno.\n\n"
        "Príklad 2:\n"
        "Komentár: \"Ďakujem ti za pomoc, veľmi si si to cením.\"\n"
        "Odpoveď: Nie.\n\n"
        "Príklad 3:\n"
        "Komentár: \"Zasa trepeš hlúposti ako vždy.\"\n"
        "Odpoveď: Áno.\n\n"
        "Príklad 4:\n"
        "Komentár: \"Prajem pekný deň.\"\n"
        "Odpoveď: Nie.\n\n"
        f"Komentár: \"{text}\"\n"
        "Odpoveď:"
    )

# Pridanie niekoľkých príkladov do promptu zlepšilo konzistenciu odpovedí
# Skúšal som aj viac ako 5 príkladov, ale potom už prompt presahoval tokenový limit






## 3. Volanie OpenAI API

In [ ]:
results = []

for i, row in df.iterrows():
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": create_prompt(row["text"])}],
            temperature=0
        )
        sentiment = response.choices[0].message.content.strip()
    except Exception:
        sentiment = "CHYBA"
    results.append({"text": row["text"], "true_label": row["label"], "gpt_sentiment": sentiment})
    if i % 10 == 0:
        print(f"Spracovaných {i} komentárov...")

pd.DataFrame(results).to_csv("sentiment_results_test_1000_fewshot.csv", index=False)
print("Výsledky uložené.")


## 4. Čistenie odpovedí a výpočet predikcií

In [ ]:
df = pd.read_csv("sentiment_results_test_1000_fewshot.csv")

# Vyčistenie odpovedí modelu
df["gpt_sentiment_clean"] = (
    df["gpt_sentiment"]
    .str.lower()
    .str.replace(".", "", regex=False)
    .str.strip()
)

df["predicted_label"] = df["gpt_sentiment_clean"].apply(
    lambda x: 1 if "áno" in x else (0 if "nie" in x else -1) # Pomocou lambda funkcie priraďujem binárny štítok, ak výstup obsahuje slovo "ano", označím ho ako 1 (toxický)
)

# Uloženie
df.to_csv("results_with_predictions_fewshot.csv", index=False)

## 5. Vyhodnotenie výsledkov

In [ ]:
y_true = df["true_label"]
y_pred = df["predicted_label"]

print("Klasifikačná správa:")
print(classification_report(y_true, y_pred, target_names=["Netoxický", "Toxický"]))

print(" Presnosť (accuracy):", round(accuracy_score(y_true, y_pred), 2))

print(" Precision:")
print("  Mikro:", round(precision_score(y_true, y_pred, average='micro'), 2))
print("  Makro:", round(precision_score(y_true, y_pred, average='macro'), 2))

print(" Recall:")
print("  Mikro:", round(recall_score(y_true, y_pred, average='micro'), 2))
print("  Makro:", round(recall_score(y_true, y_pred, average='macro'), 2))

print(" F1-score:")
print("  Mikro:", round(f1_score(y_true, y_pred, average='micro'), 2))
print("  Makro:", round(f1_score(y_true, y_pred, average='macro'), 2))

report_dict = classification_report(y_true, y_pred, target_names=["Netoxický", "Toxický"], output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_csv("classification_report_fewshot.csv")


report = classification_report(y_true, y_pred, target_names=["Netoxický", "Toxický"])
with open("classification_report_fewshot.txt", "w") as f:
    f.write(report)

## 6. Vizualizácia

In [ ]:
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Netoxický", "Toxický"], yticklabels=["Netoxický", "Toxický"])
plt.xlabel("Predikovaný štítok")
plt.ylabel("Skutočný štítok")
plt.title("Confusion matrix – Few-shot prompting")
plt.tight_layout()
plt.savefig("confusion_matrix_fewshot.png")
plt.show()
             